In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time

from bs4 import BeautifulSoup
from selenium import webdriver

import requests
import urllib.request
import urllib.error
import urllib.parse
import re
import json
#import math
#import datetime
#import os
#import sys
import datetime

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 경제언론사 뉴스 헤드라인 크롤링
- 한국경제
    - https://news.naver.com/main/list.nhn?mode=LPOD&mid=sec&oid=015
- 헤럴드경제
    - https://news.naver.com/main/list.nhn?mode=LPOD&mid=sec&oid=016

## 구조 분석
- `oid` = 언론사 id
- `date` = 8자리 YYYYMMDD
- `page` = 해당 date 일자의 헤드라인 뉴스 페이지
    - 1페이지
        - https://news.naver.com/main/list.nhn?mode=LPOD&mid=sec&oid=015&listType=title&date=20210203&page=1
    - 1000페이지 url 접근 시, 마지막 페이지로 이동
        - https://news.naver.com/main/list.nhn?mode=LPOD&mid=sec&oid=015&listType=title&date=20210203&page=1000


In [56]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36'}

In [ ]:
<a href="https://news.naver.com/main/read.nhn?mode=LS2D&amp;mid=shm&amp;sid1=101&amp;sid2=260&amp;oid=417&amp;aid=0000654332">
								 2·4 대책 84만가구 건설에 주택업계 환영 "적극 협력할 것"
								</a>

In [33]:
start_date = datetime.datetime(2020, 1, 1)
yyyy = str(start_date.year)
mm = (str(start_date.month) if start_date.month > 9 else '0'+str(start_date.month))
dd = (str(start_date.day) if start_date.day > 9 else '0'+str(start_date.day))
YYYYMMDD = yyyy + mm + dd

pageNum = 1
prev_headline = ''
dates = []
headlines = []
#url_format = f'https://news.naver.com/main/list.nhn?mode=LPOD&mid=sec&oid=016&listType=title&date={YYYYMMDD}&page={pageNum}'
print(YYYYMMDD, end='/')

while YYYYMMDD != '20210101':
    
    url_format = 'https://news.naver.com/main/list.nhn?mode=LS2D&sid2=260&sid1=101&mid=shm&date={YYYYMMDD}&page={pageNum}'
    resp = requests.get(url_format, headers=headers)
    soup = BeautifulSoup(resp.text, 'html.parser')
    
    new_headlines1 = [i.text for i in soup.select("a.nclicks\(cnt_papaerart\)") if len(i.text) > 0]
    new_headlines2 = [i.text for i in soup.select("a.nclicks\(cnt_flashart\)") if len(i.text) > 0]
    new_headlines = new_headlines1 + new_headlines2

    if len(new_headlines) == 0:
        new_headlines = ['']
    
    # 마지막 페이지 판단
    if new_headlines[0] != prev_headline:
        dates += [YYYYMMDD for _ in range(len(new_headlines))]
        headlines += new_headlines
        pageNum += 1
        prev_headline = new_headlines[0]
        
    else:
        start_date += datetime.timedelta(days=1)
        yyyy = str(start_date.year)
        mm = (str(start_date.month) if start_date.month > 9 else '0'+str(start_date.month))
        dd = (str(start_date.day) if start_date.day > 9 else '0'+str(start_date.day))
        YYYYMMDD = yyyy + mm + dd
        print(YYYYMMDD, end='/')
        pageNum = 1
        prev_headline = new_headlines[0]

20200101/20200102/20200103/20200104/20200105/20200106/20200107/20200108/20200109/20200110/20200111/20200112/20200113/20200114/20200115/20200116/20200117/20200118/20200119/20200120/20200121/20200122/20200123/20200124/20200125/20200126/20200127/20200128/20200129/20200130/20200131/20200201/20200202/20200203/20200204/20200205/20200206/20200207/20200208/20200209/20200210/20200211/20200212/20200213/20200214/20200215/20200216/20200217/20200218/20200219/20200220/20200221/20200222/20200223/20200224/20200225/20200226/20200227/20200228/20200229/20200301/20200302/20200303/20200304/20200305/20200306/20200307/20200308/20200309/20200310/20200311/20200312/20200313/20200314/20200315/20200316/20200317/20200318/20200319/20200320/20200321/20200322/20200323/20200324/20200325/20200326/20200327/20200328/20200329/20200330/20200331/20200401/20200402/20200403/20200404/20200405/20200406/20200407/20200408/20200409/20200410/20200411/20200412/20200413/20200414/20200415/20200416/20200417/20200418/20200419/20200420/2

In [34]:
len(dates), len(headlines)

(153884, 153884)

In [35]:
pd.DataFrame({'date':dates, 'headline':headlines}).to_csv('./data/stock_prediction5_herald_economy.csv',
                                                         encoding='utf-8-sig')

# New Crawling
- 네이버 > 경제 > 부동산
- 2020.01.01 ~ 2020.12.31
- start url
    - https://news.naver.com/main/list.nhn?mode=LS2D&sid2=260&sid1=101&mid=shm&date=20200101&page=1

In [143]:
start_date = datetime.datetime(2020, 1, 1)
yyyy = str(start_date.year)
mm = (str(start_date.month) if start_date.month > 9 else '0'+str(start_date.month))
dd = (str(start_date.day) if start_date.day > 9 else '0'+str(start_date.day))
YYYYMMDD = yyyy + mm + dd

pageNum = 1
prev_headline = ''
dates = []
headlines = []
times = []
#url_format = f'https://news.naver.com/main/list.nhn?mode=LPOD&mid=sec&oid=016&listType=title&date={YYYYMMDD}&page={pageNum}'
print(YYYYMMDD, end='/')

while YYYYMMDD != '20210101':
    
#     url_format = 'https://news.naver.com/main/list.nhn?mode=LS2D&sid2=260&sid1=101&mid=shm&date={YYYYMMDD}&page={pageNum}'
#     resp = requests.get(url_format, headers=headers)
#     soup = BeautifulSoup(resp.text, 'html.parser')
    
#     new_headlines1 = [i.text for i in soup.select("a.nclicks\(cnt_papaerart\)") if len(i.text) > 0]
#     new_headlines2 = [i.text for i in soup.select("a.nclicks\(cnt_flashart\)") if len(i.text) > 0]
#     new_headlines = new_headlines1 + new_headlines2

    url_format = f'https://news.naver.com/main/list.nhn?mode=LS2D&sid2=260&sid1=101&mid=shm&date={YYYYMMDD}&page={pageNum}'
    resp = requests.get(url_format, headers=headers)
    soup = BeautifulSoup(resp.text, 'lxml')
    first_soup = soup.select('.newsflash_body dt:not(.photo) > a')
    #links_ = [a['href'] for a in first_soup]
    texts_ = [a.text for a in first_soup]
    dates_ = [a.text for a in soup.select('span.date')]
    dates_date_ = [date.split(' ')[0].replace('.', '') for date in dates_]
    dates_time_ = [' '.join(date.split(' ')[1:]) for date in dates_]
    

    # 마지막 페이지 판단 - 마지막 페이지 아님 & 데이터 정상 수집됨
    # 저장하고 페이지를 넘긴다
    if (texts_[0] != prev_headline) and (len(dates_) == len(texts_)):
        dates += dates_date_
        headlines += texts_
        times += dates_time_
        
        pageNum += 1
        prev_headline = texts_[0]
        
    # 마지막 페이지이거나 정상 수집되지 않았을 때
    # 저장하지 않고 다음 날로 넘긴다
    else:
        start_date += datetime.timedelta(days=1)
        yyyy = str(start_date.year)
        mm = (str(start_date.month) if start_date.month > 9 else '0'+str(start_date.month))
        dd = (str(start_date.day) if start_date.day > 9 else '0'+str(start_date.day))
        YYYYMMDD = yyyy + mm + dd
        print(len(headlines),end=' ')
        print(YYYYMMDD, end='/')        
        pageNum = 1
        prev_headline = texts_[0]
        

20200101/114 20200102/463 20200103/694 20200104/737 20200105/872 20200106/1151 20200107/1406 20200108/1683 20200109/2057 20200110/2322 20200111/2368 20200112/2484 20200113/2789 20200114/3152 20200115/3525 20200116/3914 20200117/4149 20200118/4206 20200119/4361 20200120/4825 20200121/5223 20200122/5656 20200123/5898 20200124/5976 20200125/6022 20200126/6079 20200127/6205 20200128/6441 20200129/6801 20200130/7176 20200131/7432 20200201/7471 20200202/7575 20200203/7934 20200204/8296 20200205/8587 20200206/8892 20200207/9104 20200208/9145 20200209/9245 20200210/9569 20200211/9860 20200212/10219 20200213/10648 20200214/10925 20200215/10964 20200216/11081 20200217/11410 20200218/11735 20200219/12069 20200220/12555 20200221/12829 20200222/12866 20200223/12967 20200224/13269 20200225/13575 20200226/13918 20200227/14336 20200228/14590 20200229/14632 20200301/14751 20200302/15068 20200303/15400 20200304/15704 20200305/16026 20200306/16292 20200307/16340 20200308/16458 20200309/16748 20200310/170

In [144]:
len(dates), len(headlines), len(times)

(113151, 113151, 113151)

In [145]:
pd.DataFrame({'date':dates, 'time':times, 'headline':headlines}).to_csv('./data/naver_news_real_state_2020.csv',
                                                         encoding='utf-8-sig')

# Second - 생활경제

In [146]:
start_date = datetime.datetime(2020, 1, 1)
yyyy = str(start_date.year)
mm = (str(start_date.month) if start_date.month > 9 else '0'+str(start_date.month))
dd = (str(start_date.day) if start_date.day > 9 else '0'+str(start_date.day))
YYYYMMDD = yyyy + mm + dd

pageNum = 1
prev_headline = ''
dates = []
headlines = []
times = []
#url_format = f'https://news.naver.com/main/list.nhn?mode=LPOD&mid=sec&oid=016&listType=title&date={YYYYMMDD}&page={pageNum}'
print(YYYYMMDD, end='/')

while YYYYMMDD != '20210101':
    
#     url_format = 'https://news.naver.com/main/list.nhn?mode=LS2D&sid2=260&sid1=101&mid=shm&date={YYYYMMDD}&page={pageNum}'
#     resp = requests.get(url_format, headers=headers)
#     soup = BeautifulSoup(resp.text, 'html.parser')
    
#     new_headlines1 = [i.text for i in soup.select("a.nclicks\(cnt_papaerart\)") if len(i.text) > 0]
#     new_headlines2 = [i.text for i in soup.select("a.nclicks\(cnt_flashart\)") if len(i.text) > 0]
#     new_headlines = new_headlines1 + new_headlines2

    url_format = f'https://news.naver.com/main/list.nhn?mode=LS2D&sid2=310&sid1=101&mid=shm&date={YYYYMMDD}&page={pageNum}'
    resp = requests.get(url_format, headers=headers)
    soup = BeautifulSoup(resp.text, 'lxml')
    first_soup = soup.select('.newsflash_body dt:not(.photo) > a')
    #links_ = [a['href'] for a in first_soup]
    texts_ = [a.text for a in first_soup]
    dates_ = [a.text for a in soup.select('span.date')]
    dates_date_ = [date.split(' ')[0].replace('.', '') for date in dates_]
    dates_time_ = [' '.join(date.split(' ')[1:]) for date in dates_]
    

    # 마지막 페이지 판단 - 마지막 페이지 아님 & 데이터 정상 수집됨
    # 저장하고 페이지를 넘긴다
    if (texts_[0] != prev_headline) and (len(dates_) == len(texts_)):
        dates += dates_date_
        headlines += texts_
        times += dates_time_
        
        pageNum += 1
        prev_headline = texts_[0]
        
    # 마지막 페이지이거나 정상 수집되지 않았을 때
    # 저장하지 않고 다음 날로 넘긴다
    else:
        start_date += datetime.timedelta(days=1)
        yyyy = str(start_date.year)
        mm = (str(start_date.month) if start_date.month > 9 else '0'+str(start_date.month))
        dd = (str(start_date.day) if start_date.day > 9 else '0'+str(start_date.day))
        YYYYMMDD = yyyy + mm + dd
        print(len(headlines),end=' ')
        print(YYYYMMDD, end='/')        
        pageNum = 1
        prev_headline = texts_[0]
        

20200101/113 20200102/473 20200103/707 20200104/727 20200105/855 20200106/1161 20200107/1424 20200108/1771 20200109/2090 20200110/2257 20200111/2281 20200112/2376 20200113/2662 20200114/2884 20200115/3161 20200116/3496 20200117/3711 20200118/3750 20200119/3937 20200120/4257 20200121/4547 20200122/4767 20200123/4918 20200124/4967 20200125/5003 20200126/5044 20200127/5160 20200128/5372 20200129/5674 20200130/5943 20200131/6118 20200201/6156 20200202/6247 20200203/6544 20200204/6844 20200205/7142 20200206/7469 20200207/7717 20200208/7755 20200209/7845 20200210/8148 20200211/8416 20200212/8759 20200213/9207 20200214/9411 20200215/9433 20200216/9546 20200217/9820 20200218/10098 20200219/10387 20200220/10758 20200221/10931 20200222/10963 20200223/11065 20200224/11317 20200225/11595 20200226/11915 20200227/12285 20200228/12543 20200229/12563 20200301/12704 20200302/13001 20200303/13319 20200304/13646 20200305/14002 20200306/14223 20200307/14249 20200308/14371 20200309/14662 20200310/14963 202

In [149]:
len(dates), len(headlines), len(times)

(98490, 98490, 98490)

In [150]:
pd.DataFrame({'date':dates, 'time':times, 'headline':headlines}).to_csv('./data/naver_news_living_economy_2020.csv',
                                                         encoding='utf-8-sig')

In [80]:

url_format = f'https://news.naver.com/main/list.nhn?mode=LS2D&sid2=260&sid1=101&mid=shm&date=20201229&page=6'
resp = requests.get(url_format, headers=headers)
soup = BeautifulSoup(resp.text, 'html.parser')
first_soup = soup.select('.newsflash_body dt:not(.photo) > a')
#links_ = [a['href'] for a in first_soup]
texts_ = [a.text for a in first_soup]
dates_ = [a.text for a in soup.select('span.date')]
dates_date_ = [date.split(' ')[0].replace('.', '') for date in dates_]
dates_time_ = [' '.join(date.split(' ')[1:]) for date in dates_]

In [85]:
first_soup

[<a href="https://news.naver.com/main/read.nhn?mode=LS2D&amp;mid=shm&amp;sid1=101&amp;sid2=260&amp;oid=028&amp;aid=0002526306">
 								 변창흠 장관 “설 이전에 서울 도심 추가 공급 대책 내놓겠다”
 								</a>,
 <a href="https://news.naver.com/main/read.nhn?mode=LS2D&amp;mid=shm&amp;sid1=101&amp;sid2=260&amp;oid=011&amp;aid=0003848749">
 								 변창흠, "설 이전 공급대책 마련... 개발이익 공유시스템도 구축"
 								</a>,
 <a href="https://news.naver.com/main/read.nhn?mode=LS2D&amp;mid=shm&amp;sid1=101&amp;sid2=260&amp;oid=023&amp;aid=0003586491">
 								 변창흠 “설 명절 전 도심 주택 확대방안 발표”
 								</a>,
 <a href="https://news.naver.com/main/read.nhn?mode=LS2D&amp;mid=shm&amp;sid1=101&amp;sid2=260&amp;oid=277&amp;aid=0004819344">
 								 변창흠 "서울에 충분한 주택 공급 가능… 설 이전에 공급방안 발표"
 								</a>,
 <a href="https://news.naver.com/main/read.nhn?mode=LS2D&amp;mid=shm&amp;sid1=101&amp;sid2=260&amp;oid=018&amp;aid=0004817222">
 								 [전문]변창흠 국토부 장관 "내년 설 전에 25번째 부동산대책 발표"
 								</a>,
 <a href="https://news.naver.com/main/read.nhn?mode=LS2D&a

### 

In [152]:
start_date = datetime.datetime(2020, 1, 1)
yyyy = str(start_date.year)
mm = (str(start_date.month) if start_date.month > 9 else '0'+str(start_date.month))
dd = (str(start_date.day) if start_date.day > 9 else '0'+str(start_date.day))
YYYYMMDD = yyyy + mm + dd

pageNum = 1
prev_headline = ''
dates = []
headlines = []
times = []
#url_format = f'https://news.naver.com/main/list.nhn?mode=LPOD&mid=sec&oid=016&listType=title&date={YYYYMMDD}&page={pageNum}'
print(YYYYMMDD, end='/')

while YYYYMMDD != '20210101':
    
#     url_format = 'https://news.naver.com/main/list.nhn?mode=LS2D&sid2=260&sid1=101&mid=shm&date={YYYYMMDD}&page={pageNum}'
#     resp = requests.get(url_format, headers=headers)
#     soup = BeautifulSoup(resp.text, 'html.parser')
    
#     new_headlines1 = [i.text for i in soup.select("a.nclicks\(cnt_papaerart\)") if len(i.text) > 0]
#     new_headlines2 = [i.text for i in soup.select("a.nclicks\(cnt_flashart\)") if len(i.text) > 0]
#     new_headlines = new_headlines1 + new_headlines2

    url_format = f'https://news.naver.com/main/list.nhn?mode=LS2D&sid2=258&sid1=101&mid=shm&date={YYYYMMDD}&page={pageNum}'
    resp = requests.get(url_format, headers=headers)
    soup = BeautifulSoup(resp.text, 'lxml')
    first_soup = soup.select('.newsflash_body dt:not(.photo) > a')
    #links_ = [a['href'] for a in first_soup]
    texts_ = [a.text for a in first_soup]
    dates_ = [a.text for a in soup.select('span.date')]
    dates_date_ = [date.split(' ')[0].replace('.', '') for date in dates_]
    dates_time_ = [' '.join(date.split(' ')[1:]) for date in dates_]
    

    # 마지막 페이지 판단 - 마지막 페이지 아님 & 데이터 정상 수집됨
    # 저장하고 페이지를 넘긴다
    if (texts_[0] != prev_headline) and (len(dates_) == len(texts_)):
        dates += dates_date_
        headlines += texts_
        times += dates_time_
        
        pageNum += 1
        prev_headline = texts_[0]
        
    # 마지막 페이지이거나 정상 수집되지 않았을 때
    # 저장하지 않고 다음 날로 넘긴다
    else:
        start_date += datetime.timedelta(days=1)
        yyyy = str(start_date.year)
        mm = (str(start_date.month) if start_date.month > 9 else '0'+str(start_date.month))
        dd = (str(start_date.day) if start_date.day > 9 else '0'+str(start_date.day))
        YYYYMMDD = yyyy + mm + dd
        print(len(headlines),end=' ')
        print(YYYYMMDD, end='/')        
        pageNum = 1
        prev_headline = texts_[0]
        

20200101/133 20200102/1494 20200103/2501 20200104/2577 20200105/2741 20200106/3940 20200107/5275 20200108/6555 20200109/8165 20200110/9312 20200111/9398 20200112/9560 20200113/10987 20200114/12238 20200115/13494 20200116/14625 20200117/15662 20200118/15740 20200119/15919 20200120/17222 20200121/18533 20200122/20072 20200123/21198 20200124/21425 20200125/21529 20200126/21592 20200127/21912 20200128/23341 20200129/24801 20200130/26395 20200131/27816 20200201/27902 20200202/28055 20200203/29629 20200204/31273 20200205/32827 20200206/34717 20200207/36015 20200208/36091 20200209/36241 20200210/37648 20200211/39257 20200212/40754 20200213/42311 20200214/43639 20200215/43714 20200216/43884 20200217/44994 20200218/46210 20200219/47884 20200220/49481 20200221/50760 20200222/50861 20200223/51032 20200224/52517 20200225/53774 20200226/55237 20200227/56870 20200228/58244 20200229/58351 20200301/58528 20200302/59821 20200303/61192 20200304/62637 20200305/64059 20200306/65250 20200307/65358 20200308

In [153]:
len(dates), len(headlines), len(times)

(345441, 345441, 345441)

In [154]:
pd.DataFrame({'date':dates, 'time':times, 'headline':headlines}).to_csv('./data/naver_news_security_market_2020.csv',
                                                         encoding='utf-8-sig')